In [1]:
# 오버샘플링 모델 로드(원핫인코딩 미적용 - 성능문제로 구글코랩에서 모델 제작 및 정확도 테스트 완료)
# 필요 패키지 로드
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fonm
from matplotlib import font_manager, rc
mpl.rcParams['axes.unicode_minus'] = False
font_path = 'C:/Windows/Fonts/정직_1.ttf'
font = fonm.FontProperties(fname=font_path).get_name()
rc('font', family=font)
import pandas as pd
import seaborn as sns

In [2]:
# KNN모델 테스트
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib

In [3]:
# 기존에 처리한 데이터 전처리 코드 로드
df = pd.read_csv('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트(21.9.27~10.6)\\Project-1\\조정범\\Data\\모델제작용데이터셋(헬스케어).csv')
for i in df.columns:
    df[i] = df[i].astype('category')

print(f"전체환자 데이터 수 : {len(df)} / 입원환자 데이터 수 : {len(df[df['서식코드'] == 1])} / 비입원환자 데이터 수 : {len(df[df['서식코드'] == 0])}")

# 출처 : https://g0n1.tistory.com/34
# 클래스 불균형 처리를 해주는 imbalanced-learn를 호출
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# ros, rus 객체
ros = RandomOverSampler()
rus = RandomUnderSampler()

# Over 샘플링
oversampled_data, oversampled_label = ros.fit_resample(df[['성별코드', '연령대코드', '시도코드', '진료과목코드', '주상병코드', '부상병코드','심결가산율']].copy(), df[['서식코드']].copy())
oversampled_data = pd.DataFrame(oversampled_data, columns = ['성별코드', '연령대코드', '시도코드', '진료과목코드', '주상병코드', '부상병코드','심결가산율'])
oversampled_data['서식코드'] = oversampled_label

df_train1 = oversampled_data[['서식코드']].copy() # 테스트로 사용할 컬럼들
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis=1)
        data = data.drop(column, axis=1)
    return data
dummy_columns = ['서식코드']
df_train1_dummy = dummy_data(oversampled_data, dummy_columns)
print(f"전체환자 데이터 수 : {len(oversampled_data)} / 입원환자 데이터 수 : {len(oversampled_data[oversampled_data['서식코드'] == 1])} / 비입원환자 데이터 수 : {len(oversampled_data[oversampled_data['서식코드'] == 0])}")
X = oversampled_data.iloc[:, :-1]
y = oversampled_data.iloc[:, -1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 7)

전체환자 데이터 수 : 13178345 / 입원환자 데이터 수 : 283962 / 비입원환자 데이터 수 : 12894383
전체환자 데이터 수 : 25788766 / 입원환자 데이터 수 : 12894383 / 비입원환자 데이터 수 : 12894383


In [4]:
model = joblib.load('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트(21.9.27~10.6)\\Project-1\\조정범\\Data\\머신러닝모델\\K-Nearest_Neighbor Model\\KNN모델(over_원핫인코딩no).pkl')

C:\Users\Bestc\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [12]:
# 입력시 받아줄 딕셔너리 작성
# 성별 딕셔너리
sex_dict = dict(zip(['남성', '여성'], [1, 2])) 
print(sex_dict)

# 연령대 코드 딕셔너리
age_code_list = ['00~04', '05~09', '10~14', '15~19', '20~24', '25~29', '30~34', '35~39', '40~44', '45~49', '50~54', '55~59', '60~64', 
                 '65~69', '70~74', '75~79', '80~84', '85~89']
age_dict = dict(zip(age_code_list, [i for i in range(1, 19)]))
print(age_dict)

# -------------------- 수정 및 재작성필요 * 데이터 전처리 및 시각화 필드 코드참조하여 수정중 -------------------- 
# 시도코드 딕셔너리 - 사용할 geojson파일과 명칭 일치화 완료
sido_table = pd.read_csv('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트(21.9.27~10.6)\\Project-1\\조정범\\Data\\시도코드테이블.csv')
sido_dict = dict(zip(sido_table['시도명'], sido_table['시도코드']))
print(sido_dict)

# 서식코드 딕셔너라
form_code_list = ['청구서', '의과입원', '의과외래', '치과입원', '치과외래', '조산원입원', '보건기관입원', '보건기관외래',
                  '정신과 낮 병동', '정신과 입원', '정신과 외래', '한방입원', '한방외래', '직접조제', '처방조제', '조산원외래']
form_dict = dict(zip(form_code_list, [_ for _ in range(1, len(form_code_list) - 3)] + [13, 20, 21, 99]))
print(form_dict)

# 진료과목코드 딕셔너리
# 진료과목 코드 활용을 위한 전처리(1) / 파일출처 : 보건의료빅데이터개방시스템 (https://opendata.hira.or.kr/op/opc/selectColumnCodeList.do?pageIndex=8)
df_dsbjt= pd.read_excel('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트(21.9.27~10.6)\\Project-1\\조정범\\Data\\컬럼정보_코드.xls')
df_dsbjt_train = df_dsbjt[df_dsbjt['코드구분'] == '진료과목코드'].drop(['코드구분', '설명'], axis = 1)
df_dsbjt_train['코드'] = df_dsbjt_train['코드'].apply(lambda x: int(x))  
dsbjt_dict = dict(zip(df_dsbjt_train['명칭'], df_dsbjt_train['코드']))
print(dsbjt_dict)

# # 상병코드 딕셔너리
eng_list = ['A','B','C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
code_dict = dict(zip(eng_list, [i for i in range(len(eng_list))]))
print(code_dict)

# 심결가산율 카테고리화 및 인덱싱
df1 = pd.read_csv('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트(21.9.27~10.6)\\Project-1\\조정범\\Data\\헬스케어데이터.csv')
rate_list = pd.DataFrame(df1['심결가산율'].value_counts()).index
rate_list = sorted(rate_list)
rate_dict = dict(zip(rate_list, [i for i in range(len(rate_list))]))
print(rate_dict)

{'남성': 1, '여성': 2}
{'00~04': 1, '05~09': 2, '10~14': 3, '15~19': 4, '20~24': 5, '25~29': 6, '30~34': 7, '35~39': 8, '40~44': 9, '45~49': 10, '50~54': 11, '55~59': 12, '60~64': 13, '65~69': 14, '70~74': 15, '75~79': 16, '80~84': 17, '85~89': 18}
{'강원도': 42, '경기도': 41, '경상남도': 48, '경상북도': 47, '광주광역시': 29, '대구광역시': 27, '대전광역시': 30, '부산광역시': 26, '서울특별시': 11, '세종특별자치시': 36, '울산광역시': 31, '인천광역시': 28, '전라남도': 46, '전라북도': 45, '제주특별자치도': 49, '충청남도': 44, '충청북도': 43}
{'청구서': 1, '의과입원': 2, '의과외래': 3, '치과입원': 4, '치과외래': 5, '조산원입원': 6, '보건기관입원': 7, '보건기관외래': 8, '정신과 낮 병동': 9, '정신과 입원': 10, '정신과 외래': 11, '한방입원': 12, '한방외래': 13, '직접조제': 20, '처방조제': 21, '조산원외래': 99}
{'일반의': 0, '내과': 1, '신경과': 2, '정신건강의학과': 3, '외과': 4, '정형외과': 5, '신경외과': 6, '흉부외과': 7, '성형외과': 8, '마취통증의학과': 9, '산부인과': 10, '소아청소년과': 11, '안과': 12, '이비인후과': 13, '피부과': 14, '비뇨의학과': 15, '영상의학과': 16, '방사선종양학과': 17, '병리과': 18, '진단검사의학과': 19, '결핵과': 20, '재활의학과': 21, '핵의학과': 22, '가정의학과': 23, '응급의학과': 24, '직업환경의학과': 25, '예방의학과': 26, '기타1(치과)': 27,

In [8]:
df.columns

Index(['성별코드', '연령대코드', '시도코드', '서식코드', '진료과목코드', '주상병코드', '부상병코드', '심결가산율'], dtype='object')

In [ ]:
# 순서 : '성별코드', '연령대코드', '시도코드', '진료과목코드', '주상병코드', '부상병코드', '심결가산율'
value_list = ['남성', '60~64', '전라남도', '신경외과', 'A', 'D', '0.25']
value_list[0]
input_list = [sex_dict[value_list[0]]]

In [5]:
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy : {accuracy*100}')